In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
#Load data
train_data = pd.read_csv('D:/Users/Maria/Downloads/training_data/training_data.csv', delimiter=';',decimal=',')
test_data = pd.read_csv('D:/Users/Maria/Downloads/test_data_no_target/test_data_no_target.csv', delimiter=';', decimal=',')

In [3]:
#Preprocessing data
train_data.replace('NA', np.nan, inplace=True)
train_data.replace('', np.nan, inplace=True)
train_data = train_data.drop('Group', axis=1)
cols = train_data.columns
train_data[cols] = train_data[cols].apply(pd.to_numeric, errors='coerce')
train_data[cols] = train_data[cols].fillna(train_data[cols].mean())
#Split data into features and target
X_train = train_data.drop(columns=['Class', 'Perform'])
y_train = train_data['Class']


In [4]:
test_data.replace('NA', np.nan, inplace=True)
test_data.replace('', np.nan, inplace=True)
test_data = test_data.drop('Group', axis=1)
cols = test_data.columns
test_data[cols] = test_data[cols].apply(pd.to_numeric, errors='coerce')
test_data[cols] = test_data[cols].fillna(test_data[cols].mean())

In [5]:
#Preprocessing pipeline for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Fit and transform the training data
X_train_preprocessed = numerical_transformer.fit_transform(X_train)

# Preprocess the test data
X_test_preprocessed = numerical_transformer.transform(test_data)

c:\Users\Maria\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\Maria\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Maria\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\Maria\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:757: FutureWarning:

In [6]:
#Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [49, 50, 51],  #After some experiments I found out that 48 is not changing
    'max_depth': [9, 10, 11],
    'min_samples_split': [10,11,12],
    'min_samples_leaf': [1, 2, 3]
}

#RF Classifier
rf_classifier = RandomForestClassifier(random_state=2000)

#Look for the best parameters
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid,
                           cv=15, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(X_train_preprocessed, y_train)

best_params = grid_search.best_params_
print(best_params)

Fitting 15 folds for each of 81 candidates, totalling 1215 fits
{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 11, 'n_estimators': 51}


In [7]:
#Training
best_rf_classifier = RandomForestClassifier(**best_params, random_state=42)
best_rf_classifier.fit(X_train_preprocessed, y_train)

#Getting predictions
y_test_pred = best_rf_classifier.predict(X_test_preprocessed)
np.savetxt('test_predictions_rf.txt', y_test_pred, fmt='%d')

In [8]:
#CV
cv_scores = cross_val_score(best_rf_classifier, X_train_preprocessed, y_train, cv=15, scoring='accuracy')
print(cv_scores)

[0.50374532 0.47003745 0.46816479 0.49812734 0.49812734 0.49155722
 0.48030019 0.46341463 0.50281426 0.48968105 0.47654784 0.49155722
 0.49718574 0.48217636 0.51031895]
